In [1]:
import ROOT

if "/home/submit/pdmonte/CMSSW_10_6_27/src/Hrare2023/analysis/func_marti.so" not in ROOT.gSystem.GetLibraries():
    ROOT.gSystem.CompileMacro("/home/submit/pdmonte/CMSSW_10_6_27/src/Hrare2023/analysis/func_marti.cc","k")

if "/home/submit/pdmonte/CMSSW_10_6_27/src/Hrare2023/analysis/functions.so" not in ROOT.gSystem.GetLibraries():
    ROOT.gSystem.CompileMacro("/home/submit/pdmonte/CMSSW_10_6_27/src/Hrare2023/analysis/functions.cc","k")

date = "OCT27"
pi1Mass = 0.13957018
k1Mass = 0.493677
mass1, mass2 = 0, 0
chainSGN_Phi3 = ROOT.TChain("events")
chainSGN_Phi3.Add("/data/submit/pdmonte/outputs/{0}/2018/outname_mc{1}_GFcat_{2}_2018.root".format(date, 1039, "Phi3Cat"))
dfSGN_Phi3 = ROOT.RDataFrame(chainSGN_Phi3)
dfSGN_Phi3 = (dfSGN_Phi3.Define("scale", "w*lumiIntegrated")
                .Define("newDitrackMass", "sum2Body(goodMeson_trk1_pt[0], goodMeson_trk1_eta[0], goodMeson_trk1_phi[0], {}, goodMeson_trk2_pt[0], goodMeson_trk2_eta[0], goodMeson_trk2_phi[0], {}).M()".format(pi1Mass, pi1Mass))
                .Define("DiffFittedMass", "goodMeson_ditrk_mass - goodMeson_ditrk_mass_GEN")
                .Define("DiffSumMass", "newDitrackMass - goodMeson_ditrk_mass_GEN")
                .Define("DiffRawMass", "goodMeson_mass_raw - goodMeson_mass_GEN")
                .Define("DiffModifiedMass", "goodMeson_mass - goodMeson_mass_GEN")
                .Define("Residual_ditrk_pt", "goodMeson_ditrk_pt - goodMeson_ditrk_pt_GEN")
                .Define("Residual_old_pt", "goodMeson_pt - goodMeson_pt_GEN")
                .Define("Residual_ditrk_mass", "goodMeson_ditrk_mass - goodMeson_ditrk_mass_GEN")
                .Define("Residual_ditrk_eta", "goodMeson_ditrk_eta - goodMeson_ditrk_eta_GEN")
                .Define("Residual_ditrk_phi", "goodMeson_ditrk_phi - goodMeson_ditrk_phi_GEN"))

Welcome to JupyROOT 6.28/00


In [239]:
defaultHistColors = [ROOT.kRed + 1, ROOT.kBlue, ROOT.kGreen + 2, ROOT.kYellow + 1, ROOT.kOrange + 8]

ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetPadTickX(1)
ROOT.gStyle.SetPadTickY(1)

def savePlot(histograms, options=None):
    cs = ROOT.TCanvas("canvas", "canvas", 900, 900)
    xlow, ylow, xup, yup = 0.60, 0.70, 0.87, 0.87
    if options["data"]:
        ydiv = 0.20
        deltadiv = 0.05
        pad1 = ROOT.TPad("upper_pad", "", 0., ydiv, 1., 1.)#xlow, ylow, xup, yup
        pad1.SetTopMargin(0.125)
        pad1.Draw()
        pad2 = ROOT.TPad("lower_pad", "", 0., 0., 1., ydiv + deltadiv)
        pad2.Draw()
        ylow, yup = 0.625, 0.8375
    else:
        pad1 = ROOT.TPad("upper_pad", "", 0., 0., 1., 1.)#xlow, ylow, xup, yup
        pad1.Draw()
    if "HCandMass" in options:
        xlow, xup = 0.13, 0.40
    legend4 = ROOT.TLegend(xlow, ylow, xup, yup)
    

    stack4 = ROOT.THStack()
    

    usedColors = options["colors"] if "colors" in options else defaultHistColors

    markers = []
    maxHeight = 0
    for i, h in enumerate(histograms):
        maxHeight = max(maxHeight, h[1].GetMaximum())
        if "style" in options:
            if options["style"][i] == "l":
                stack4.Add(h[1])
                legend4.AddEntry(h[1], h[0], "l")
                h[1].SetLineWidth(3)
                h[1].SetLineColor(usedColors[i])
            elif options["style"][i] == "f":
                legend4.AddEntry(h[1], h[0], "f")
                stack4.Add(h[1])
                h[1].SetLineWidth(0)
                h[1].SetFillColor(usedColors[i])
            elif options["style"][i] == "p":
                legend4.AddEntry(h[1], h[0], "lep")
                markers.append((h[1], usedColors[i]))
        else:
            stack4.Add(h[1])
            legend4.AddEntry(h[1], h[0], "l")
            h[1].SetLineWidth(3)
            h[1].SetLineColor(usedColors[i])

    pad1.cd()# Draw onto main plot

    if "HCandMass" in options:
        hStack = histograms[0][1].Clone("allstack")
        hStack.Add(histograms[1][1])
        hStack.Add(histograms[2][1])
        maxHeight = max(maxHeight, hStack.GetMaximum())
        print(maxHeight)
        stack4.Draw("hist")
        line1 = ROOT.TLine(115., 0., 115., maxHeight*10)
        line1.SetLineColor(11)
        line1.Draw()
        line2 = ROOT.TLine(135., 0., 135., maxHeight*10)
        line2.SetLineColor(11)
        line2.Draw()
    else:
        stack4.Draw("hist nostack")

    for h, col in markers:
        h.SetMarkerStyle(20)
        h.SetMarkerSize(1.3)
        h.SetLineWidth(3)
        h.SetMarkerColor(col)
        h.SetLineColor(col)
        h.Draw("EP SAME")
        
    stack4.SetMaximum(1.2*maxHeight)

    if "labelXAxis" in options:
        stack4.GetXaxis().SetTitle(options["labelXAxis"])
    if "labelYAxis" in options:
        stack4.GetYaxis().SetTitle(options["labelYAxis"])
    if "xRange" in options:
        stack4.GetXaxis().SetRangeUser(options["xRange"][0], options["xRange"][1])
    if "logScale" in options:
        if options["logScale"]:
            stack4.SetMaximum(10*maxHeight)
            stack4.SetMinimum(1)
            pad1.SetLogy(1)
            
    
    if not options["data"]:
        legend4.SetTextFont(42)
        legend4.SetFillStyle(0)
        legend4.SetBorderSize(0)
        legend4.SetTextSize(0.04)
        legend4.SetTextAlign(12)
        legend4.Draw("same")

        text = ROOT.TLatex()
        text.SetNDC()
        text.SetTextFont(72)
        text.SetTextSize(0.045)
        text.DrawLatex(0.105, 0.913, "CMS")
        text.SetTextFont(42)
        text.DrawLatex(0.105 + 0.12, 0.913, "Internal" if options["data"] else "Simulation")
        text.SetTextSize(0.035)
        text.DrawLatex(0.59, 0.913, "#sqrt{{s}} = 13 TeV, {lumi} fb#kern[{space}]{{^{{-1}}}}".format(lumi=39.54, space=-0.8 if options["data"] else -0.1))
        stack4.GetXaxis().SetTitleSize(0.040)
        stack4.GetXaxis().SetLabelSize(0.035)
        stack4.GetYaxis().SetTitleSize(0.040)
        stack4.GetYaxis().SetLabelSize(0.035)
        stack4.GetYaxis().SetTitleOffset(1.35)

    else:
        fact = 1.25
        legend4.SetTextFont(42)
        legend4.SetFillStyle(0)
        legend4.SetBorderSize(0)
        legend4.SetTextSize(0.04*fact)
        legend4.SetTextAlign(12)
        legend4.Draw("same")

        text = ROOT.TLatex()
        text.SetNDC()
        text.SetTextFont(72)
        text.SetTextSize(0.045*fact)
        text.DrawLatex(0.105, 0.89125, "CMS")
        text.SetTextFont(42)
        text.DrawLatex(0.105 + 0.12, 0.89125, "Internal" if options["data"] else "Simulation")
        text.SetTextSize(0.035*fact)
        text.DrawLatex(0.59, 0.89125, "#sqrt{{s}} = 13 TeV, {lumi} fb#kern[{space}]{{^{{-1}}}}".format(lumi=39.54, space=-0.1 if options["data"] else -0.1))
        
        stack4.GetXaxis().SetLabelOffset(99)#0.005 default
        stack4.GetXaxis().SetTitleOffset(99)#1 is default
        stack4.GetXaxis().SetTitleSize(0.040*fact)
        stack4.GetXaxis().SetLabelSize(0.035*fact)
        stack4.GetYaxis().SetTitleSize(0.040*fact)
        stack4.GetYaxis().SetLabelSize(0.035*fact)
        stack4.GetYaxis().SetTitleOffset(1.08)
        #draw ratio
        pad2.cd()
        pad2.SetTopMargin(0.0)
        pad2.SetBottomMargin(0.4)
        
        hData = histograms[-1][1]
        mcBKG = histograms[0][1]
        ratio = hData.Clone("dataratio")
        print("ALL mcTOT integral(): ",mcBKG.Integral())
        print("ALL data integral(): ", hData.Integral())

        ratio.Divide(mcBKG)
        ratio.GetYaxis().SetTitle("data/MC")
        ratio.GetYaxis().SetRangeUser(0.5,1.5)
        ratio.GetYaxis().SetTitleOffset(0.3)
        ratio.GetYaxis().SetTitleSize(0.15)
        ratio.GetYaxis().SetLabelSize(0.12)
        ratio.GetYaxis().SetNdivisions(5, 2, 0)
        fact = 4.0
        ratio.GetXaxis().SetTitleSize(0.040*fact)
        ratio.GetXaxis().SetLabelSize(0.035*fact)
        ratio.GetXaxis().SetLabelOffset(0.005*fact)#0.005 default
        ratio.GetXaxis().SetTitleOffset(1)#1 is default
        ratio.GetXaxis().SetTitle(options["labelXAxis"])
        
        ratio.SetTitle("")

        ratio.Draw("pe")
        lineZero = ROOT.TLine(mcBKG.GetXaxis().GetXmin(), 1.,  mcBKG.GetXaxis().GetXmax(), 1.)
        lineZero.SetLineColor(11)
        lineZero.Draw("same")

    cs.SaveAs("/home/submit/pdmonte/public_html/thesisPlots/test{}.png".format("" if options["data"] else "1"))

In [240]:
df = dfSGN_Phi3
data = True
options = {"labelXAxis": "m_{{#gamma{}}} [GeV]".format("#phi"), "labelYAxis": "Events", "style": ["f", "f", "p"], "colors": [ROOT.kOrange - 9, ROOT.kRed + 2, ROOT.kBlack], "data": data}
nbins, xlow, xhigh = 100, 0, 1.2
histograms = []
name1 = "signal"
h1 = df.Histo1D(("hist", name1, nbins, xlow, xhigh), "goodMeson_ditrk_mass_GEN", "scale").GetValue()
name2 = "backgriund"
h2 = df.Histo1D(("hist", name2, nbins, xlow, xhigh), "goodMeson_ditrk_mass_GEN", "scale").GetValue()
name3 = "data"
h3 = df.Histo1D(("hist", name3, nbins, xlow, xhigh), "goodMeson_ditrk_mass_GEN", "scale").GetValue()
h1.Scale(1.5)
h2.Scale(1.2)
h3.Scale(1.8)
histograms.append((name1, h1))
histograms.append((name2, h2))
histograms.append((name3, h3))
a = savePlot(histograms, options=options)

ALL mcTOT integral():  32808.20894148018
ALL data integral():  39369.85072977622


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file /home/submit/pdmonte/public_html/thesisPlots/test.png has been created
